In [1]:
!pip install -q datasets
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# LoRA Layer

In [2]:
# class Linear(nn.Linear, LoRALayer):
#     # LoRA implemented in a dense layer
#     def __init__(
#         self,
#         in_features: int,
#         out_features: int,
#         r: int = 0,
#         lora_alpha: int = 1,
#         lora_dropout: float = 0.,
#         fan_in_fan_out: bool = False, # Set this to True if the layer to replace stores weight like (fan_in, fan_out)
#         merge_weights: bool = True,
#         **kwargs
#     ):
#         nn.Linear.__init__(self, in_features, out_features, **kwargs)
#         LoRALayer.__init__(self, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout,
#                            merge_weights=merge_weights)

#         self.fan_in_fan_out = fan_in_fan_out
#         # Actual trainable parameters
#         if r > 0:
#             self.lora_A = nn.Parameter(self.weight.new_zeros((r, in_features)))
#             self.lora_B = nn.Parameter(self.weight.new_zeros((out_features, r)))
#             self.scaling = self.lora_alpha / self.r
#             # Freezing the pre-trained weight matrix
#             self.weight.requires_grad = False
#         self.reset_parameters()
#         if fan_in_fan_out:
#             self.weight.data = self.weight.data.transpose(0, 1)

#     def reset_parameters(self):
#         nn.Linear.reset_parameters(self)
#         if hasattr(self, 'lora_A'):
#             # initialize B the same way as the default for nn.Linear and A to zero
#             # this is different than what is described in the paper but should not affect performance
#             nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
#             nn.init.zeros_(self.lora_B)

#     def train(self, mode: bool = True):
#         def T(w):
#             return w.transpose(0, 1) if self.fan_in_fan_out else w
#         nn.Linear.train(self, mode)
#         if mode:
#             if self.merge_weights and self.merged:
#                 # Make sure that the weights are not merged
#                 if self.r > 0:
#                     self.weight.data -= T(self.lora_B @ self.lora_A) * self.scaling
#                 self.merged = False
#         else:
#             if self.merge_weights and not self.merged:
#                 # Merge the weights and mark it
#                 if self.r > 0:
#                     self.weight.data += T(self.lora_B @ self.lora_A) * self.scaling
#                 self.merged = True

#     def forward(self, x: torch.Tensor):
#         def T(w):
#             return w.transpose(0, 1) if self.fan_in_fan_out else w
#         if self.r > 0 and not self.merged:
#             result = F.linear(x, T(self.weight), bias=self.bias)
#             result += (self.lora_dropout(x) @ self.lora_A.transpose(0, 1) @ self.lora_B.transpose(0, 1)) * self.scaling
#             return result
#         else:
#             return F.linear(x, T(self.weight), bias=self.bias)

# 1. 한국어 데이터셋

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

df = pd.read_json('/content/drive/MyDrive/공ㅂ/train_data.json')

In [5]:
df

,instructions,output
0,예.,그~ 선생님은
1,그~ 선생님은,혹시 이제 결혼하셨는데
2,혹시 이제 결혼하셨는데,어~ 혹시 뭐 이상형이 어떻게 되셨나요?
3,어~ 혹시 뭐 이상형이 어떻게 되셨나요?,네. 저는
4,네. 저는,어~
...,...,...
13909,뭘 먹을까?,기승전 고기 아니겠습니까? 무한리필 고기집 추천합니다.
13910,"근데, 바로 근무해야 하는데 좀 부담되네.",그럼 몸보신을 위해서 추어탕 어떻습니까?
13911,"그래, 나름 괜찮네.",그럼요. 괜찮은 선택을 했다고 생각하실 거예요.
13912,오늘은 날이 좋네.,점심은 밖에서 맛있는거 사드세요!


In [6]:
import datasets

data = datasets.load_dataset('json', data_files='/content/drive/MyDrive/공ㅂ/train_data.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['instructions', 'output'],
        num_rows: 13914
    })
})

In [8]:
data = data.map(
    lambda x: {'text': f"{ x['instructions']}\n\n 정답: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/13914 [00:00<?, ? examples/s]

# 2. Tokenizer

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00026-of-00028.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/518M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/13914 [00:00<?, ? examples/s]

In [ ]:
data['train'][0]['text']

'예.\n\n 정답: 그~ 선생님은<|endoftext|>'

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)

NameError: name 'model' is not defined

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

In [ ]:
# GPU 할당이 어떻게 되어있는지 체킹하여 봅니다.
for i in model.named_parameters():
    print(f"{i[0]} -> {i[1].device}")

base_model.model.gpt_neox.embed_in.weight -> cuda:0
base_model.model.gpt_neox.layers.0.input_layernorm.weight -> cuda:0
base_model.model.gpt_neox.layers.0.input_layernorm.bias -> cuda:0
base_model.model.gpt_neox.layers.0.post_attention_layernorm.weight -> cuda:0
base_model.model.gpt_neox.layers.0.post_attention_layernorm.bias -> cuda:0
base_model.model.gpt_neox.layers.0.attention.query_key_value.base_layer.weight -> cuda:0
base_model.model.gpt_neox.layers.0.attention.query_key_value.base_layer.bias -> cuda:0
base_model.model.gpt_neox.layers.0.attention.query_key_value.ia3_l.default -> cuda:0
base_model.model.gpt_neox.layers.0.attention.dense.weight -> cuda:0
base_model.model.gpt_neox.layers.0.attention.dense.bias -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.weight -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.bias -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_4h_to_h.base_layer.weight -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_4h_

In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token


trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        num_train_epochs=4,
        max_steps=15930, # 가장 최적화된 15930 step까지 훈련되었습니다.
        learning_rate=5e-5,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim='adamw_torch'),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,6.453500
20,6.322400
30,6.942900
40,6.489000
50,6.553200
60,6.469800
70,5.994000
80,6.455700
90,6.887700
100,6.473100
